In [1]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def download_file(url, folder_path):
    response = requests.get(url, stream=True)
    file_name = os.path.join(folder_path, os.path.basename(urlparse(url).path))
    
    with open(file_name, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)

def scrape_and_download(base_url, folder_path, subjects):
    for subject in subjects:
        subject_url = f"{base_url}S{subject:03d}/"
        response = requests.get(subject_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all links with .edf extension
        links = soup.find_all('a', href=lambda href: (href and href.endswith('.edf.event')))

        for link in links:
            file_url = urljoin(subject_url, link['href'])
            download_file(file_url, folder_path)

if __name__ == "__main__":
    base_url = "https://archive.physionet.org/pn4/eegmmidb/"
    folder_path = "downloaded_files_event"
    subjects_range = range(1, 110)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    scrape_and_download(base_url, folder_path, subjects_range)
